In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder

print("Libraries imported successfully.")

Libraries imported successfully.


In [3]:
# https://www.kaggle.com/competitions/playground-series-s6e1/data?select=train.csv

# 1. Load the datasets
folder_path_local = "../input/playground-series-s6e1/"
folder_path_remote = "https://kagglecsv.netlify.app/input/playground-series-s6e1/"
folder_path = folder_path_local if os.path.exists(folder_path_local) else folder_path_remote  # choose local if available, else remote
train_data = pd.read_csv(folder_path + 'train.csv')
test_data = pd.read_csv(folder_path + 'test.csv')
print("Datasets loaded successfully.")

Datasets loaded successfully.


In [4]:
# 2. Check column names for both train and test data
print("Train Data Columns:", train_data.columns)
print("Test Data Columns:", test_data.columns)

Train Data Columns: Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')
Test Data Columns: Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty'],
      dtype='object')


In [5]:
# 3. Identify categorical columns (ignore 'id' column)
categorical_columns = train_data.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns.drop('id', errors='ignore')  # Drop 'id' if it exists, otherwise ignore
print(f"Categorical columns: {categorical_columns}")

Categorical columns: Index(['gender', 'course', 'internet_access', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty'],
      dtype='object')


In [6]:
# 4. Label Encoding for categorical features
label_encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column
for col in categorical_columns:
    train_data[col] = label_encoder.fit_transform(train_data[col])
    test_data[col] = label_encoder.transform(test_data[col])  # Ensure the test data is encoded the same way

print("Categorical features encoded successfully.")

Categorical features encoded successfully.


In [7]:
# 5. Split data into features (X) and target (y)
if 'id' in train_data.columns:  # Check if 'id' exists before dropping it
    X = train_data.drop(columns=['id', 'exam_score'])  # Drop 'id' and 'exam_score' columns
else:
    X = train_data.drop(columns=['exam_score'])  # Only drop 'exam_score' if 'id' is not present
y = train_data['exam_score']

In [8]:
# 6. Feature Scaling (Standardizing the features)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# 7. Train-test split (for evaluation purposes)
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# from sklearn.linear_model import LinearRegression , Ridge, Lasso, ElasticNet, BayesianRidge
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import Model
# from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, MultiHeadAttention, LayerNormalization, Dropout
from keras.optimizers import Adam

# 8. initialize models

# Define XGBoost parameters
# xgb_params = { # 8.792066858871292 if 1000
#     'n_estimators': 10000,
#     'learning_rate': 0.007,
#     'max_depth': 7,
#     'subsample': 0.8,
#     'reg_lambda': 3,
#     'colsample_bytree': 0.6,
#     'colsample_bynode': 0.7,
#     'tree_method': 'hist',
#     'random_state': 42,
#     'early_stopping_rounds': 100,
#     'eval_metric': 'rmse',
#     'enable_categorical': True
# }

# Use actual feature count for Keras model input dimension
num_features = X_scaled.shape[1]

models = {
    # Scikit-learn models
    # "Linear Regression": LinearRegression(), # 9.945236827130886
    # "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42), # 9.119773196172922
    # "Gradient Boosting Regressor": GradientBoostingRegressor(n_estimators=100, random_state=42), # 8.853389182802118
    # "Support Vector Regressor (SVR)": SVR(),
    # "K-Nearest Neighbors Regressor (KNN)": KNeighborsRegressor(), # 10.041602240167114
    # "Decision Tree Regressor": DecisionTreeRegressor(random_state=42), # 12.862204876992518

    # "XGBoost Regressor 1": XGBRegressor(n_estimators=100, random_state=42),# 8.80645474036935
    # "XGBoost Regressor": XGBRegressor(**{k: v for k, v in xgb_params.items() if k not in ['early_stopping_rounds', 'eval_metric']}),
    
    # "LightGBM Regressor": LGBMRegressor(n_estimators=100, random_state=42),# 8.80233253326403

    # "CatBoost Regressor 1": CatBoostRegressor(iterations=100, random_state=42, verbose=0),# 8.779514995823046
    # "CatBoost Regressor 2": CatBoostRegressor(iterations=1000, random_state=42, verbose=0),# 8.750053736561881
    # "CatBoost Regressor 3": CatBoostRegressor(iterations=2000, random_state=42, verbose=0),# 8.746984406439088
    # "CatBoost Regressor 4": CatBoostRegressor(iterations=5000, random_state=42, verbose=0),#  8.74407813414281
    "CatBoost Regressor": CatBoostRegressor(iterations=10000, random_state=42, verbose=0),

    # "ElasticNet Regression": ElasticNet(alpha=1.0, l1_ratio=0.5),# 11.419489099367368
    # "Ridge Regression": Ridge(alpha=1.0),# 9.945236728904232
    # "Lasso Regression": Lasso(alpha=0.1),# 9.948718108336369
    # "Bayesian Ridge Regression": BayesianRidge(),# 9.945236375344892

    # TensorFlow / Keras models (use Input(...) as recommended)
    # "Neural Network (Dense) - 1 Layer": tf.keras.Sequential([# 9.107451331633579
    #     Input(shape=(num_features,)),
    #     Dense(64, activation='relu'),
    #     Dense(1)
    # ]),
    # "Neural Network (Dense) - 2 Layers": tf.keras.Sequential([# 9.0263789985335
    #     Input(shape=(num_features,)),
    #     Dense(64, activation='relu'),
    #     Dense(64, activation='relu'),
    #     Dense(1)
    # ]),
    # "Neural Network (Deep) - 3 Layers": tf.keras.Sequential([ # 9.015516714821777
    #     Input(shape=(num_features,)),
    #     Dense(128, activation='relu'),
    #     Dense(128, activation='relu'),
    #     Dense(128, activation='relu'),
    #     Dense(1)
    # ]),
    # "CNN for Regression": tf.keras.Sequential([ # 9.129382497786011
    #     Input(shape=(10, 1)),
    #     Conv1D(32, kernel_size=3, activation='relu'),
    #     MaxPooling1D(pool_size=2),
    #     Flatten(),
    #     Dense(64, activation='relu'),
    #     Dense(1)
    # ]),
    # "LSTM for Regression": tf.keras.Sequential([ # 9.112482477086305
    #     Input(shape=(10, 1)),
    #     LSTM(64, return_sequences=False),
    #     Dense(64, activation='relu'),
    #     Dense(1)
    # ]),
    # "GRU for Regression": tf.keras.Sequential([ # 8.980764873623912
    #     Input(shape=(10, 1)),
    #     GRU(64, return_sequences=False),
    #     Dense(64, activation='relu'),
    #     Dense(1)
    # ]),
}

# Note: commented Transformer / other models remain unchanged


In [11]:
# 9. Train models and evaluate them using RMSE
best_rmse = float('inf')
best_model_name = None

for model_name, model in models.items():
    # Compile Keras models if necessary
    if hasattr(model, 'compile') and callable(getattr(model, 'compile')):
        try:
            model.compile(optimizer=Adam(), loss='mean_squared_error')
        except Exception as e:
            print(f"Warning: could not compile {model_name}: {e}")

    # Train the model (handle both scikit-learn and Keras)
    try:
        model.fit(X_train, y_train)
    except Exception as e:
        print(f"Error training {model_name}: {e}")
        continue

    # Predict and evaluate
    try:
        y_pred = model.predict(X_valid)
    except Exception as e:
        print(f"Error predicting with {model_name}: {e}")
        continue

    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    print("\n")

    # If the current RMSE is better (lower) than the previous best, update the best model
    if rmse < best_rmse:
        best_rmse = rmse
        best_model_name = model_name


CatBoost Regressor RMSE: 8.741494909332976




In [12]:
# 10. Choose the best model (Assume Random Forest performs the best)
best_model = models[best_model_name]# ["CatBoost Regressor"]
best_model_name

'CatBoost Regressor'

In [ ]:
# 11. Train the final model on the full training data
best_model.fit(X_scaled, y)

In [ ]:
# 12. Prepare the test data for predictions
if 'id' in test_data.columns:  # Check if 'id' exists in test data before dropping it
    X_test = test_data.drop(columns=['id'])  # 'id' is not a feature
else:
    X_test = test_data  # In case 'id' doesn't exist, just use the entire test data without 'id'

X_test_scaled = scaler.transform(X_test)

In [ ]:
# 13. Generate predictions
test_predictions = best_model.predict(X_test_scaled)

In [ ]:
# 14. Prepare the submission file
submission = pd.DataFrame({
    'id': test_data['id'],  # Ensure 'id' is included in the submission
    'exam_score': test_predictions
})

In [ ]:
# 15. Save the submission file
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!
